## Project Milestone 4 - Linguistic Harbingers of Betrayal Extension

First, input all the libraries we need to get the plot:

In [1]:
import json

from scipy.stats import sem
import numpy as np
import matplotlib.pyplot as plt

Since the data file contains only the selected games, there is no need to consider the selection criteria when retreving the message data. 
*The condition "relationships that contain at least two consecutive and reciprocated acts of support that span at least three seasons in game time, with no more than five seasons passing between two acts of friendship" is fullfilled.

Here, we read the data from json file and get the number of games, which is 500, including 250 selected games ended up betrayal and 250 selected games ended up lasting friendship between two players.


In [2]:
# read data from diplomacy_data.json with reading mode
with open("diplomacy_data.json", "r") as f:
        data = json.load(f)
len(data)

500

Since we only consider dialogs that happen before the last act of friendship, we need to know when does the last act of friendship happen. Later when extracting feature values, extract them from the seasons that is before the last support season.

In [3]:
# a function to get the season where the last support between two players happened
# entry refer to one entry in the dataset, one game
def last_support(entry):
    seasons = entry['seasons']
    last_support = None
    for season in seasons[:-1]:
        if 'support' in season['interaction'].values():
            last_support = season['season']
    return last_support

Given the messages sent by a player from a season in a game, we need to calculate the different variable values of the messages.

Variables we consider:

1. sent_pos: Positive sentiment
2. sent_neu: Neutral sentiment
3. sent_neg: Negative sentiment
4. discourse_comp: Discourse complexity (calculated through comparison, contingency, expansive and temporal)
5. plan: Planning level (calculated through future)
6. argu_level: Argumentation level (calculated through claim and premise)
7. n_request: Number of requests
8. politeness: Politeness
9. subj: Subjectivity
10. n_words: Number of words
11. n_sentences: Number of sentences


In [4]:
# a function to get the average value across the seasons for all features of the messages
# msgs: messages sent by a player from a season in a game
def extract_features(msgs):    
    # count total number of messages in the conversation
    sum_politeness = 0
    for m in msgs:
        sum_politeness += m['politeness']
    
    avg_politeness = sum_politeness/len(msgs)

    return avg_politeness

From paper: To ensure that we are studying conversational patterns that occur only when the two individuals in the dyad are ostensibly being friends, we only extract features from the messages exchanged before the last act of friendship.
Therefore, the messages we extract should from seasons less than last support season.

In [5]:
# a function to get average politeness scores from dataset entries 
# data: the dataset where we extract politeness score from
# betrayal: if the games we consider end up betrayal
# betrayer: if the person to be analyzed is the (potential) betrayer
def politeness_for_plot(data, betrayal, betrayer):
    average_politeness = []
    
    # loop in every game in data
    for entry in data:
        for season in entry['seasons']:
            # consider every season in the game, extract features from the messages exchanged before the last act of friendship
            if(season['season'] <= last_support(entry)):
                if(len(season['messages']['betrayer']) > 0 and len(season['messages']['victim']) > 0):
                    if(entry['betrayal'] == betrayal):
                        if(betrayer):
                            average_politeness.append(msgs_politeness(season['messages']['betrayer']))
                        else:
                            average_politeness.append(msgs_politeness(season['messages']['victim']))
    # return value is a list of mean politeness score of the indicated player in the indicated type of games
    return average_politeness

Get the politeness score of the four conditions below:

In [6]:
# for the games end up betrayal, get politeness score for each betrayer
betray_er = politeness_for_plot(data, True, True)

# for the games end up betrayal, get politeness score for each victim
betray_vi = politeness_for_plot(data, True, False) 

# for the games not end up betrayal, get politeness score for each potential betrayer
control_er = politeness_for_plot(data, False, True) 

# for the games not end up betrayal, get politeness score for each potential victim
control_vi = politeness_for_plot(data, False, False) 